# Install Dependencies

In [1]:
#!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib

# Import Dependencies

In [2]:
import cv2
import os # abstracts operating system from our python( working with directories file paths)
import random
import numpy as np
from matplotlib import pyplot as plt

In [3]:
#plt.imshow?? shows describtion of a command

There are two class API to define a model in tf. keras. According to the doc

Sequential class: Sequential groups a linear stack of layers into a tf. keras.Model.

Model class: Model group's layers into an object with training and inference features.

In [4]:
# Import tensorflow dependencies - Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten #layer types
#Conv2d - convolutial layer
#Dense - connection layers
#MaxPooling2D-reducing amount of pixels of the image by pulling mulitple inot one
# Input - what we are passing to our model
# Flatten - takes information form previous layers and flattens them into single dimenasion
import tensorflow as tf

C:\Users\ortek\anaconda3\envs\tf_gpu\lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.20.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Set GPU Growth

In [5]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU') #Acessing all gpus on machine
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True) #attempts to allocate only as much GPU memory based on 
                                                        #runtime allocations, it starts out allocating very little memory, 
                                                        #and as sessions get run and more GPU memory is needed, we extend
                                                        #the GPU memory region needed by the TensorFlow process.

# Create folder Structures

In [6]:
# Setup paths
POS_PATH = os.path.join('data', 'positive') # positive verification
NEG_PATH = os.path.join('data', 'negative') # negative verification
ANC_PATH = os.path.join('data', 'anchor') # input image

In [7]:
# Make the directories
#os.makedirs(POS_PATH)
#os.makedirs(NEG_PATH)
#os.makedirs(ANC_PATH)

# 2. Collect Positives and Anchors

# Untar Labelled Faces in the Wild Dataset

In [8]:
# http://vis-www.cs.umass.edu/lfw/

In [9]:
# Uncompress Tar GZ Labelled Faces in the Wild Dataset
!tar -xf lfw.tgz

In [10]:
# Move LFW Images to the following repository data/negative
for directory in os.listdir('lfw'): # Looping thru folders inside Lfw
    for file in os.listdir(os.path.join('lfw', directory)):# Looping thru every file in sub-directory
        EX_PATH = os.path.join('lfw', directory, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)

# Collect Positive and Anchor Classes

In [11]:
# Import uuid library to generate unique image names
import uuid

In [12]:
os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))

'data\\anchor\\20fba80d-7ee3-11ee-be58-a4f933cc0852.jpg'

In [13]:
# # Establish a connection to the webcam
# cap = cv2.VideoCapture(0)#choose camera
# while cap.isOpened(): #Loop thru frames in web cam
#     ret, frame = cap.read() # Read capture at the point in time
   
#      #Cut down frame to 250x250px
#     frame = frame[120:120+250,200:200+250, :]
    
# #     # Collect anchors 
#     if cv2.waitKey(1) & 0XFF == ord('a'):
#         # Create the unique file path 
#         imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
#         # Write out anchor image
#         cv2.imwrite(imgname, frame)
    
#     # Collectq positives
#     if cv2.waitKey(1) & 0XFF == ord('p'):
#         # Create the unique file path 
#         imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
#         # Write out positive image
#         cv2.imwrite(imgname, frame)
    
#     # Show image back to screen
#     cv2.imshow('Image Collection', frame)
    
#     # Breaking gracefully
#     if cv2.waitKey(1) & 0XFF == ord('q'):
#         break
        
# # Release the webcam
# cap.release()
# # Close the image show frame
# cv2.destroyAllWindows()

# 3. Load and Preprocess Images

# Get image Directories

In [14]:
anchor = tf.data.Dataset.list_files(ANC_PATH+'\*.jpg').take(300) # list files grabs all the data from a directory
positive = tf.data.Dataset.list_files(POS_PATH+'\*.jpg').take(300)# it creates a pipeline
negative = tf.data.Dataset.list_files(NEG_PATH+'\*.jpg').take(300)

In [15]:
dir_test = anchor.as_numpy_iterator()

In [16]:
dir_test.next()

b'data\\anchor\\a50ee55d-7a37-11ee-9379-a4f933cc0852.jpg'

# Preprocessing - Scale and Resize

In [17]:
def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    # Preprocessing steps - resizing the image to be 100x100x3
    img = tf.image.resize(img, (100,100))
    # Scale image to be between 0 and 1 
    img = img / 255.0

    # Return image
    return img

In [18]:
img = preprocess(b'data\\anchor\\a327833f-7a37-11ee-953d-a4f933cc0852.jpg')

In [19]:
img

<tf.Tensor: shape=(100, 100, 3), dtype=float32, numpy=
array([[[0.6338235 , 0.64166665, 0.5867647 ],
        [0.6343137 , 0.6401961 , 0.5879902 ],
        [0.6394608 , 0.6401961 , 0.59166664],
        ...,
        [0.5833333 , 0.58137256, 0.50539213],
        [0.5735294 , 0.577451  , 0.5147059 ],
        [0.57254905, 0.5772059 , 0.50441176]],

       [[0.6313726 , 0.6431373 , 0.5745098 ],
        [0.63039213, 0.6401961 , 0.57254905],
        [0.63161767, 0.63553923, 0.5708333 ],
        ...,
        [0.57254905, 0.57867646, 0.50147057],
        [0.56789213, 0.5732843 , 0.5004902 ],
        [0.5632353 , 0.57303923, 0.4852941 ]],

       [[0.6343137 , 0.6460784 , 0.5715686 ],
        [0.63186276, 0.6431373 , 0.5688726 ],
        [0.63308823, 0.63848037, 0.56715685],
        ...,
        [0.5683824 , 0.5781863 , 0.5090686 ],
        [0.56985295, 0.58137256, 0.4990196 ],
        [0.56397057, 0.57965684, 0.48161766]],

       ...,

       [[0.6068627 , 0.65367645, 0.5860294 ],
        [0.60

# Create Labelled Dataset

In [20]:
# (anchor, positive) => 1,1,1,1,1
# (anchor, negative) => 0,0,0,0,0

In [21]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [22]:
samples = data.as_numpy_iterator()

In [23]:
example = samples.next()

In [24]:
example

(b'data\\anchor\\a46a8fd1-7a37-11ee-ae72-a4f933cc0852.jpg',
 b'data\\positive\\eb02f217-7a37-11ee-af49-a4f933cc0852.jpg',
 1.0)

# Build Train and Test Partition

In [25]:
def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

In [26]:
res = preprocess_twin(*example)

In [27]:
# Build dataloader pipeline
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=10000)

In [28]:
samples = data.as_numpy_iterator()
sam=samples.next()

In [29]:
#plt.imshow(sam[1])

In [30]:
sam[2]

0.0

In [31]:
# Training partition
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8) # starrts pre-processing next batch of images to avoid bottleneck

In [32]:
train_samples= train_data.as_numpy_iterator()

In [33]:
train_sample = train_samples.next()

In [34]:
train_sample

(array([[[[0.6333333 , 0.64117646, 0.5862745 ],
          [0.63039213, 0.6392157 , 0.58137256],
          [0.6284314 , 0.6401961 , 0.5735294 ],
          ...,
          [0.5715686 , 0.5754902 , 0.50490195],
          [0.56985295, 0.5737745 , 0.5031863 ],
          [0.5735294 , 0.5852941 , 0.5107843 ]],
 
         [[0.63529414, 0.64411765, 0.5862745 ],
          [0.63014704, 0.64191175, 0.5752451 ],
          [0.62328434, 0.63504905, 0.5683824 ],
          ...,
          [0.57916665, 0.58504903, 0.51936275],
          [0.56936276, 0.5745098 , 0.50147057],
          [0.5627451 , 0.56764704, 0.49411765]],
 
         [[0.6306372 , 0.64240193, 0.5757353 ],
          [0.62916666, 0.64093137, 0.5742647 ],
          [0.6357843 , 0.64754903, 0.5808824 ],
          ...,
          [0.57058823, 0.58235294, 0.5137255 ],
          [0.5713235 , 0.5801471 , 0.4997549 ],
          [0.56960785, 0.577451  , 0.49215686]],
 
         ...,
 
         [[0.622549  , 0.6519608 , 0.6019608 ],
          [0.62769

In [35]:
len(train_sample[0])

16

In [36]:
# Testing partition
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In [37]:
train_samples

# 4. Model Engineering

In [38]:
inp = Input(shape=(100,100,3), name='input_image')

In [39]:
inp

<KerasTensor: shape=(None, 100, 100, 3) dtype=float32 (created by layer 'input_image')>

In [40]:
c1 = Conv2D(64, (10,10), activation='relu')(inp) # 64 channels, filter shape = 10x10 pixels, stride = 1
c1

<KerasTensor: shape=(None, 91, 91, 64) dtype=float32 (created by layer 'conv2d')>

In [41]:
m1 = MaxPooling2D((2,2), padding='same')(c1)# 64 units, shape of (2,2)
m1

<KerasTensor: shape=(None, 46, 46, 64) dtype=float32 (created by layer 'max_pooling2d')>

In [42]:
def make_embedding(): 
    inp = Input(shape=(100,100,3), name='input_image') # Tensorflow layers
    
    # First block
    c1 = Conv2D(64, (10,10), activation='relu')(inp) # 64 filters, filter shape = 10x10 pixels, stride = 1
    m1 = MaxPooling2D(pool_size=(2, 2), padding='same')(c1)# 64 units, shape of (2,2)
    
    # Second block
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(pool_size=(2, 2), padding='same')(c2)
    
    # Third block 
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(pool_size=(2, 2), padding='same')(c3)
    
    # Final embedding block
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4) # 9216
    d1 = Dense(4096, activation='sigmoid')(f1)
    
    
    return Model(inputs=[inp], outputs=[d1], name='embedding')

In [43]:
embedding = make_embedding()

In [44]:
embedding.summary()

Model: "embedding"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_image (InputLayer)     [(None, 100, 100, 3)]     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 91, 91, 64)        19264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 46, 46, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 40, 40, 128)       401536    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 20, 20, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 17, 17, 128)       262272    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 9, 9, 128)         0 

# Build Distance Layer

In [45]:
# Siamese L1 Distance class
class L1Dist(Layer): # tensorflow.keras.layers.Layer
    
    # Init method - inheritance
    def __init__(self, **kwargs): #__init__() function is called automatically every time the class is being used to create a new object.,
        super().__init__()        #kwargs - keyword argument - dictionary, super() function will make the child class inherit all the methods and properties from its parent:
        
       
    #  similarity calculation
    def call(self, input_embedding, validation_embedding):#we will combine this in dense layer
        return tf.math.abs(input_embedding - validation_embedding)

In [46]:
l1 = L1Dist()

# Make Siamese model

In [47]:
input_image = Input(name='input_img', shape=(100,100,3))
validation_image = Input(name='validation_img', shape=(100,100,3))

In [48]:
inp_embedding = embedding(input_image)
val_embedding = embedding(validation_image)
inp_embedding

<KerasTensor: shape=(None, 4096) dtype=float32 (created by layer 'embedding')>

In [49]:
val_embedding

<KerasTensor: shape=(None, 4096) dtype=float32 (created by layer 'embedding')>

In [50]:
siamese_layer = L1Dist()
distances = siamese_layer(inp_embedding,val_embedding) #Vector with 4096 units, This represents distance between validation and input embedding
distances

<KerasTensor: shape=(None, 4096) dtype=float32 (created by layer 'l1_dist_1')>

In [51]:
classifier = Dense(1, activation='sigmoid')(distances) #Combining distances into sigmoid 
classifier

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense_1')>

In [52]:
siamese_network = Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [53]:
siamese_network.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_img (InputLayer)          [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
validation_img (InputLayer)     [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
embedding (Functional)          (None, 4096)         38960448    input_img[0][0]                  
                                                                 validation_img[0][0]             
__________________________________________________________________________________________________
l1_dist_1 (L1Dist)              (None, 4096)         0           embedding[0][0]     

In [54]:
def make_siamese_model(): 
    
    # Anchor image input in the network
    input_image = Input(name='input_img', shape=(100,100,3))
    
    # Validation image in the network 
    validation_image = Input(name='validation_img', shape=(100,100,3))
    
    # Combine siamese distance components
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))
    
    # Classification layer 
    classifier = Dense(1, activation='sigmoid')(distances) #Combining distances into sigmoid 
    
    return Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork') # Model from tf

In [55]:
siamese_model = make_siamese_model()

In [56]:
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_img (InputLayer)          [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
validation_img (InputLayer)     [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
embedding (Functional)          (None, 4096)         38960448    input_img[0][0]                  
                                                                 validation_img[0][0]             
__________________________________________________________________________________________________
distance (L1Dist)               (None, 4096)         0           embedding[2][0]     

# 5 Training

# Setup Loss and Optimizer

In [57]:
# Loss function
binary_cross_loss = tf.losses.BinaryCrossentropy() # from_logits = True if inputs are not normalized

In [58]:
opt = tf.keras.optimizers.Adam(1e-4) # 0.0001

# Checkpoints

In [59]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt') # model.load('path') will load pretrained weights into existing model
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

# Build Train Setup Function

In [60]:
test_batch = train_data.as_numpy_iterator()

In [61]:
batch_1 = test_batch.next()

In [62]:
len(batch_1[0]) # How many images we have in our batch
#batch_1[0]-anchor images
#batch_1[1]-positive/negatives images
#batch_1[0]-labels of images

16

In [63]:
X = batch_1[:2]
np.array(X).shape

(2, 16, 100, 100, 3)

In [64]:
@tf.function # Compiles a function into a callable Tensorflow graph
def train_step(batch):
    
    # Record all of our operations 
    with tf.GradientTape() as tape: # tf.GradientTape - Record operations for automatic differentation, allowing backpropagation
        # Get anchor and positive/negative image
        X = batch[:2]
        # Get label
        y = batch[2]
        
        # Forward pass
        yhat = siamese_model(X, training=True) # Certain Layers will only activate when this parameter is set to true
        # Calculate loss
        loss = binary_cross_loss(y, yhat) # bce(y_true, y_pred).numpy()
    print(loss)
        
    # Calculate gradients
    grad = tape.gradient(loss, siamese_model.trainable_variables) # calculating gradients foa all trainable variables
    
    # Calculate updated weights and apply to siamese model
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables)) # Optimizer is calculating and propagating new weights using
                                                                      #Adam's optimisation algorithm
    # Return loss
    return loss

# Training loop

In [65]:
# Import metric calculations
from tensorflow.keras.metrics import Precision, Recall

In [75]:
def train(data, EPOCHS):
    # Loop through epochs
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data)) # if used before in a model class no need for that
        
        # Creating a metric object 
        r = Recall()
        p = Precision()
        
        # Loop through each batch
        for idx, batch in enumerate(data):
            # Run train step here
            loss = train_step(batch)
            yhat = siamese_model.predict(batch[:2])
            r.update_state(batch[2], yhat)
            p.update_state(batch[2], yhat) 
            progbar.update(idx+1)
        print('loss:',loss.numpy(),'result Recall:' ,r.result().numpy(),'Precission:', p.result().numpy())
        
        # Save checkpoints
        if epoch % 10 == 0: 
            checkpoint.save(file_prefix=checkpoint_prefix)

# Training a model

In [76]:
EPOCHS=50

In [77]:
train(train_data,EPOCHS)


 Epoch 1/50
27/27 [==============================] - 1s 42ms/step
loss: 2.2351824e-06 result Recall: 1.0 Precission: 1.0

 Epoch 2/50
27/27 [==============================] - 1s 40ms/step
loss: 7.450582e-08 result Recall: 1.0 Precission: 1.0

 Epoch 3/50
27/27 [==============================] - 1s 41ms/step
loss: 8.940819e-06 result Recall: 1.0 Precission: 1.0

 Epoch 4/50
27/27 [==============================] - 1s 41ms/step
loss: 7.4505886e-07 result Recall: 1.0 Precission: 1.0

 Epoch 5/50
27/27 [==============================] - 1s 41ms/step
loss: 5.3644237e-07 result Recall: 1.0 Precission: 1.0

 Epoch 6/50
27/27 [==============================] - 1s 41ms/step
loss: 5.960465e-08 result Recall: 1.0 Precission: 1.0

 Epoch 7/50
27/27 [==============================] - 1s 41ms/step
loss: -0.0 result Recall: 1.0 Precission: 1.0

 Epoch 8/50
27/27 [==============================] - 1s 41ms/step
loss: 4.4703518e-07 result Recall: 1.0 Precission: 1.0

 Epoch 9/50
27/27 [================

# 6. Evaluate Model

In [79]:
len(test_data.as_numpy_iterator().next()[0])

16

In [80]:
# Get a batch of test data
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

In [81]:
y_true

array([1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1.],
      dtype=float32)

In [82]:
y_hat = siamese_model.predict([test_input, test_val])
y_hat

array([[1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [5.2532730e-08],
       [1.8868558e-09],
       [2.2384423e-08],
       [1.0000000e+00],
       [1.0000000e+00],
       [9.9999988e-01],
       [4.7538366e-12],
       [9.9999678e-01],
       [3.1131173e-10],
       [2.5423122e-07],
       [1.5657388e-09],
       [1.0000000e+00],
       [9.9999988e-01]], dtype=float32)

In [83]:
# Post processing the results 
[1 if prediction > 0.5 else 0 for prediction in y_hat ]

[1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1]

# Calculate metrics

In [85]:
# Creating a metric object 
m = Recall()

# Calculating the recall value 
m.update_state(y_true, y_hat)

# Return Recall Result
m.result().numpy()

1.0

In [86]:
# Creating a metric object 
m = Precision()

# Calculating the recall value 
m.update_state(y_true, y_hat)

# Return Recall Result
m.result().numpy()

1.0

In [ ]:
r = Recall()
p = Precision()

for test_input, test_val, y_true in test_data.as_numpy_iterator():
    yhat = siamese_model.predict([test_input, test_val])
    r.update_state(y_true, yhat)
    p.update_state(y_true,yhat) 

print(r.result().numpy(), p.result().numpy())

# Visualize Results

In [99]:
# # Set plot size 
# plt.figure(figsize=(10,8))

# # Set first subplot
# plt.subplot(1,2,1)
# #plt.imshow(test_input[3])

# # Set second subplot
# plt.subplot(1,2,2)
# #plt.imshow(test_val[3])

# # Renders cleanly
# #plt.show()

# Save model

In [89]:
#Save weights
siamese_model.save('siamesemodelv2.h5')

In [90]:
siamese_model = tf.keras.models.load_model('siamesemodelv2.h5', #L1Dist distance layer
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [91]:
# Make predictions with reloaded model
siamese_model.predict([test_input, test_val])

array([[1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [5.2532730e-08],
       [1.8868558e-09],
       [2.2384423e-08],
       [1.0000000e+00],
       [1.0000000e+00],
       [9.9999988e-01],
       [4.7538366e-12],
       [9.9999678e-01],
       [3.1131173e-10],
       [2.5423122e-07],
       [1.5657388e-09],
       [1.0000000e+00],
       [9.9999988e-01]], dtype=float32)

In [92]:
# View model summary
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_img (InputLayer)          [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
validation_img (InputLayer)     [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
embedding (Functional)          (None, 4096)         38960448    input_img[0][0]                  
                                                                 validation_img[0][0]             
__________________________________________________________________________________________________
l1_dist_3 (L1Dist)              (None, 4096)         0           embedding[0][0]     

# 7 Real time

In [ ]:
application_data\verification_images

In [ ]:
os.listdir(os.path.join('application_data', 'verification_images'))

In [ ]:
os.path.join('application_data', 'input_image', 'input_image.jpg')

In [100]:
for image in os.listdir(os.path.join('application_data', 'verification_images')):
    validation_img = os.path.join('application_data', 'verification_images', image)
    print(validation_img)

application_data\verification_images\d3a57fc5-7a37-11ee-9079-a4f933cc0852 — kopia.jpg
application_data\verification_images\d3f45c96-7a37-11ee-b52e-a4f933cc0852 — kopia.jpg
application_data\verification_images\d3ff1120-7a37-11ee-af48-a4f933cc0852 — kopia.jpg
application_data\verification_images\d513ba82-7a37-11ee-afa7-a4f933cc0852 — kopia.jpg
application_data\verification_images\d58dead3-7a37-11ee-b056-a4f933cc0852.jpg
application_data\verification_images\d65f912d-7a37-11ee-8138-a4f933cc0852.jpg
application_data\verification_images\d66bda14-7a37-11ee-bcce-a4f933cc0852.jpg
application_data\verification_images\d68a0ca9-7a37-11ee-9c2f-a4f933cc0852.jpg
application_data\verification_images\d6f18d76-7a37-11ee-a493-a4f933cc0852 — kopia.jpg
application_data\verification_images\d7a8b3c4-7a37-11ee-a04e-a4f933cc0852 — kopia.jpg
application_data\verification_images\d8d9c089-7a37-11ee-aefc-a4f933cc0852.jpg
application_data\verification_images\dbb60ca6-7a37-11ee-b346-a4f933cc0852 — kopia.jpg
applicat

In [102]:
result = siamese_model.predict(list(np.expand_dims([ validation_img], axis=1)))

ValueError: in user code:

    C:\Users\ortek\anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py:1586 predict_function  *
        return step_function(self, iterator)
    C:\Users\ortek\anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\ortek\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\ortek\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\ortek\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\ortek\anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\training

In [163]:
def verify(model, detection_threshold, verification_threshold):
    # Build results array
    results = []
    input_img = preprocess(os.path.join('application_data', 'input_image', 'input_image.jpg')) # preprocess data that comes from webcam
    for image in os.listdir(os.path.join('application_data', 'verification_images')):
        
        validation_img = preprocess(os.path.join('application_data', 'verification_images', image))# Because of using preprocess function we are going to be using tf
        
        # Make Predictions 
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
    
    # Detection Threshold: Metric above which a prediciton is considered positive 
    detection = np.sum(np.array(results) > detection_threshold)
    
    # Verification Threshold: Proportion of positive predictions / total positive samples 
    verification = detection / len(os.listdir(os.path.join('application_data', 'verification_images'))) 
    verified = verification > verification_threshold
    
    return results, verified

In [180]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[120:120+250,200:200+250, :]
    
    cv2.imshow('Verification', frame)
    
    # Verification trigger
    if cv2.waitKey(10) & 0xFF == ord('v'):
        # Save input image to application_data/input_image folder 
#         hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
#         h, s, v = cv2.split(hsv)

#         lim = 255 - 10
#         v[v > lim] = 255
#         v[v <= lim] -= 10
        
#         final_hsv = cv2.merge((h, s, v))
#         img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)

        cv2.imwrite(os.path.join('application_data', 'input_image', 'input_image.jpg'), frame)
        # Run verification
        results, verified = verify(siamese_model, 0.9, 0.7)
        print(verified)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

True
True
False
False
True
True
False
True
True
True
False
False
True
True
True
False
False
True
True
False
False
True
True
True
True
False
False
False
True
True
False
False
True
True
True


In [168]:
results

[array([[0.98288834]], dtype=float32),
 array([[0.9824081]], dtype=float32),
 array([[0.9837051]], dtype=float32),
 array([[0.98556626]], dtype=float32),
 array([[0.9378483]], dtype=float32),
 array([[0.9999347]], dtype=float32),
 array([[0.9999645]], dtype=float32),
 array([[0.99998903]], dtype=float32),
 array([[0.5745596]], dtype=float32),
 array([[0.5071233]], dtype=float32),
 array([[0.05842303]], dtype=float32),
 array([[0.51260793]], dtype=float32),
 array([[0.58717674]], dtype=float32),
 array([[0.5298772]], dtype=float32),
 array([[0.00505049]], dtype=float32),
 array([[0.21212901]], dtype=float32),
 array([[0.0164792]], dtype=float32),
 array([[0.09944442]], dtype=float32),
 array([[0.2122987]], dtype=float32),
 array([[0.03978927]], dtype=float32),
 array([[0.02477476]], dtype=float32),
 array([[0.04619284]], dtype=float32),
 array([[0.88556963]], dtype=float32),
 array([[0.89076364]], dtype=float32),
 array([[0.05845533]], dtype=float32),
 array([[0.09070522]], dtype=float3

In [178]:
np.squeeze(results) >0.5

array([False, False, False, False, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [179]:
np.sum(np.squeeze(results) >0.5)

44